# Dataset Creation - CTA/MRA/MRI

Ce notebook crée les datasets de cubes 3D pour l'entraînement du modèle.

**Étapes** :
1. Chargement des données localisateurs
2. Filtrage par modalité (CTA, MRA, MRI T1post, MRI T2)
3. Vérification de la disponibilité des données
4. Test sur un exemple
5. Extraction de cubes positifs (contenant anévrisme) et négatifs
6. Sauvegarde au format .npz

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
from tqdm import tqdm

# Import du package src
sys.path.append("../")

from src import (
    SERIES_DIR,
    TRAIN_CSV,
    TRAIN_LOCALIZERS_CSV,
    PROCESSED_DIR,
    print_config
)
from src.data import ajouter_Modality
from src.preprocessing import preprocessing_volume_and_coords
from src.visualization import show_middle_slices, show_slice_with_point

## 1. Configuration et chargement des données

In [ ]:
# Affichage de la configuration automatique
print_config()

In [ ]:
# Chargement DataFrames (chemins automatiques)
df_loc = pd.read_csv(TRAIN_LOCALIZERS_CSV)
df_train = pd.read_csv(TRAIN_CSV)

# Ajout modalité
df_loc = ajouter_Modality(df_loc, df_train)

print(f"Total localizers: {len(df_loc)}")
print(f"\nModalités:")
print(df_loc['Modality'].value_counts())

## 2. Filtrage par modalité

Choisissez la modalité à traiter (CTA, MRA, MRI T1post, MRI T2)

In [ ]:
# Sélection modalité
MODALITY = "CTA"  # Changez selon besoin: "CTA", "MRA", "MRI T1post", "MRI T2"

df_modality = df_loc[df_loc['Modality'] == MODALITY].reset_index(drop=True)
print(f"Nombre de {MODALITY}: {len(df_modality)}")

## 3. Vérification de la disponibilité des données

Avant de traiter les données, vérifions quels patients sont réellement disponibles.

In [ ]:
# Vérifier combien de patients de la modalité sont disponibles localement
available_patients = []
for i in range(len(df_modality)):
    series_uid = df_modality.iloc[i]['SeriesInstanceUID']
    patient_path = os.path.join(SERIES_DIR, series_uid)
    if os.path.exists(patient_path):
        available_patients.append(series_uid)

print(f"Patients {MODALITY} dans le CSV: {len(df_modality)}")
print(f"Patients {MODALITY} disponibles localement: {len(available_patients)}")

if len(available_patients) > 0:
    print(f"\n✓ {len(available_patients)} patients prêts pour le traitement")
else:
    print(f"\n⚠️ Aucun patient disponible localement")
    print(f"Les données DICOM doivent être téléchargées dans: {SERIES_DIR}")

## 4. Test sur un exemple

In [ ]:
# Test sur le premier patient disponible
if len(available_patients) > 0:
    # Utiliser le premier patient disponible identifié à l'étape 3
    series_uid = available_patients[0]
    patient_path = os.path.join(SERIES_DIR, series_uid)
    
    print(f"Patient sélectionné: {series_uid}")
    
    try:
        volume, aneurysm_coords = preprocessing_volume_and_coords(
            SERIES_DIR, patient_path, df_modality
        )
        
        print(f"Volume shape: {volume.shape}")
        print(f"Aneurysm coordinates: {aneurysm_coords}")
        
        # Visualisation
        show_middle_slices(volume)
        show_slice_with_point(volume, aneurysm_coords, plane="axial")
        
    except Exception as e:
        print(f"Erreur lors du traitement: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"⚠️ Aucun patient {MODALITY} disponible localement")
    print(f"Exécutez d'abord l'étape 3 pour vérifier les données disponibles.")

## 5. Création du dataset

**Note**: Les fonctions `extract_positives_cubes`, `extract_negative_cubes`, et `build_dataset_dict` 
doivent être ajoutées au package `src/` si elles ne le sont pas déjà.

In [ ]:
# TODO: Implémenter la création du dataset
# Exemple de structure attendue:

# dataset = {
#     'patient_0': {
#         'patient_ID': [...],
#         'cubes': [...],  # array (N, 48, 48, 48)
#         'labels': [...],  # array (N,)
#         'positions': [...]  # array (N, 13)
#     },
#     ...
# }

print("Dataset creation: À implémenter")

## 6. Sauvegarde

In [ ]:
# # Sauvegarde du dataset (utilise PROCESSED_DIR automatiquement)
# output_path = os.path.join(PROCESSED_DIR, f"{MODALITY}_dataset.npz")
# np.savez(output_path, **dataset)
# print(f"Dataset sauvegardé: {output_path}")